In [5]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib as plt
from sklearn.utils import shuffle

In [6]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data",
    shuffle=True,
    image_size = (256,256),
    batch_size = 32
)

Found 2152 files belonging to 3 classes.


In [7]:
class_names = dataset.class_names

In [8]:
def divide_dataset(dataset,train_size=0.8,valid_size=0.1,test_size=0.1,shuffle=True):
    
    assert(train_size + test_size + valid_size == 1)

    if shuffle:
        dataset = dataset .shuffle(100000)

    size = len(dataset)
    train_size = int (size*train_size)
    test_size = int (size*test_size)
    valid_size = int (size*valid_size)

    train = dataset.take(train_size)
    temp = dataset.skip(train_size)
    valid = temp.take(valid_size)
    test = temp.skip(valid_size)

    return train , valid , test

In [9]:
train , valid , test = divide_dataset(dataset)

In [11]:
train = train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
valid = valid.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test = test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [17]:
size_scale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [15]:
augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [21]:
model = models.Sequential([

    size_scale,
    augmentation,

    layers.Conv2D(32 , kernel_size=(3,3) , activation='relu' , input_shape=(32,256,256,3) ),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64 , kernel_size=(3,3) , activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(3,activation="softmax"),
])
model.build(input_shape = (32,256,256,3) )

In [24]:
# model.summary()

In [25]:
model.compile(
    optimizer = "adam",
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [26]:
history = model.fit(
    train,
    batch_size=32,
    validation_data=valid,
    verbose=1,
    epochs=50
)

Epoch 1/50
54/54 [==============================] - 93s 2s/step - loss: 0.9276 - accuracy: 0.4683 - val_loss: 0.8392 - val_accuracy: 0.5060
Epoch 2/50
54/54 [==============================] - 64s 1s/step - loss: 0.8722 - accuracy: 0.5651 - val_loss: 0.7368 - val_accuracy: 0.6488
Epoch 3/50
54/54 [==============================] - 64s 1s/step - loss: 0.6354 - accuracy: 0.7570 - val_loss: 0.5335 - val_accuracy: 0.8214
Epoch 4/50
54/54 [==============================] - 63s 1s/step - loss: 0.4232 - accuracy: 0.8339 - val_loss: 0.3006 - val_accuracy: 0.8690
Epoch 5/50
54/54 [==============================] - 63s 1s/step - loss: 0.2916 - accuracy: 0.8873 - val_loss: 0.1960 - val_accuracy: 0.9167
Epoch 6/50
54/54 [==============================] - 62s 1s/step - loss: 0.2616 - accuracy: 0.8950 - val_loss: 0.2366 - val_accuracy: 0.8988
Epoch 7/50
54/54 [==============================] - 63s 1s/step - loss: 0.2225 - accuracy: 0.9126 - val_loss: 0.1707 - val_accuracy: 0.9345
Epoch 8/50
54/54 [==

In [27]:
history.params

{'verbose': 1, 'epochs': 50, 'steps': 54}

In [28]:
scores = model.evaluate(test)

8/8 [==============================] - 7s 275ms/step - loss: 0.0393 - accuracy: 0.9844


In [29]:
scores

[0.039313215762376785, 0.984375]

In [33]:
import pandas as pd

In [56]:
d = [None]*48
len(d)

48

In [57]:
dk = pd.DataFrame()
dk["train_accuracy"] = history.history["accuracy"]
dk["valid_accuracy"] = history.history["val_accuracy"]
dk["train_losses"] = history.history["loss"]
dk["valid_losses"] = history.history["val_loss"]
dk["test_info"] = ["test_loss = 0.039313215762376785" , "test_accuracy = 0.984375"] + d
dk.to_csv("results.csv",index=False)

In [58]:
dk=pd.read_csv("results.csv")
dk.head()

,train_accuracy,valid_accuracy,train_losses,valid_losses,test_info
0,0.468310,0.505952,0.927617,0.839199,test_loss = 0.039313215762376785
1,0.565141,0.648810,0.872209,0.736754,test_accuracy = 0.984375
2,0.757042,0.821429,0.635447,0.533484,NaN
3,0.833920,0.869048,0.423228,0.300569,NaN
4,0.887324,0.916667,0.291574,0.195984,NaN


In [59]:
model_version = 1
model.save(f"D:\IIT-KGP\Deep Learning\Diese_classification\Results")

INFO:tensorflow:Assets written to: D:\IIT-KGP\Deep Learning\Diese_classification\Results\assets


INFO:tensorflow:Assets written to: D:\IIT-KGP\Deep Learning\Diese_classification\Results\assets
